In [1]:
from localization_scripts.imports import *
%load_ext autoreload
%autoreload 2

/home/smlm-workstation/miniconda3/envs/mvis/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
# filename = '/home/smlm-workstation/event-smlm/our_ev_smlm_recordings/MT_CL_2023-05-31_14-55-59.raw'
filename = '/home/smlm-workstation/event-smlm/our_ev_smlm_recordings/MT_CL_06_08/recording_2023-06-08_16-18-59.raw'
if os.path.basename(filename)[-4:] == ".raw":
    events = raw_events_to_array(filename).astype(
        [("x", "uint16"), ("y", "uint16"), ("p", "byte"), ("t", "uint64")]
    )

slice = events[(events["t"] < 200e6)]

In [48]:
from numba import njit, prange, types, typed, jit
from numba.typed import List
# from localization_scripts.event_array_processing import array_to_polarity_map
NUM_CORES = multiprocessing.cpu_count()

"""PROMINENCE is the prominence of the peaks in the convolved signals.
Smaller value detects more peaks, increasing the evaluation time."""
PROMINENCE = 18

"""DATASEET_FWHM is the FWHM of the PSF in the dataset in pixels."""
DATASEET_FWHM = 6

"""PEAK_TIME_THRESHOLD is the maximum time difference between two peaks in order to be considered as the same peak."""
PEAK_TIME_THRESHOLD = 40e3

"""PEAK_NEIGHBORS is the number of neighboring pixels to be considered when filtering same peaks."""
PEAK_NEIGHBORS = 7

"""ROI_RADIUS is the radius of the generated ROI in pixels."""
ROI_RADIUS = 8

events = slice
start_time = time.time()

# Get the minimum and maximum x and y coordinates
min_x = events["x"].min()
min_y = events["y"].min()
max_x = events["x"].max()
max_y = events["y"].max()

# Create coordinate lists
y_coords, x_coords = [min_y, max_y], [min_x, max_x]
coords = generate_coord_lists(y_coords[0], y_coords[1], x_coords[0], x_coords[1])

# Generate dictionaries and calculate max length
print(f"Analyzing the data using {NUM_CORES} cores... Events go brrrrrrrrrrrr!")
print(
    f"Converting events to dictionaries... Elapsed time: {time.time() - start_time:.2f} seconds"
)

dict_events, events_t_p_dict, coords, max_length = convert_to_hashmaps(slice, filename, max_x, max_y)

Analyzing the data using 24 cores... Events go brrrrrrrrrrrr!
Converting events to dictionaries... Elapsed time: 0.49 seconds


In [49]:
len(list(dict_events.keys()))

250473

In [53]:
len(lengths)

250473

In [54]:
max_length

894145

In [50]:
lengths = [(val[0] ,val[1][2][0]) for val in list(dict_events.items())]

In [52]:
lengths = np.sort(np.asarray(lengths), axis=0)
lengths[-10:]

array([[(719, 1037), 6094482],
       [(719, 1038), 6185263],
       [(719, 1039), 6976357],
       [(719, 1040), 8067932],
       [(719, 1041), 8476864],
       [(719, 1042), 8874138],
       [(719, 1043), 8956025],
       [(719, 1044), 10119824],
       [(719, 1045), 11574069],
       [(719, 1046), 14708398]], dtype=object)

In [125]:
from numba import njit, prange, types, typed, jit
from numba.typed import List

@njit(cache=True, nogil=True, fastmath=True)
def remove_coordinates(arr, my_dict, time_map):
    for y in range(arr.shape[0]):
        for x in range(arr.shape[1]):
            if arr[y, x] == 0:
                coord = (y, x)
                if coord in my_dict:
                    del my_dict[coord]
                if coord in time_map:
                    del time_map[coord]
    return my_dict, time_map

@njit(cache=True, nogil=True, fastmath=True)
def remove_coordinates_by_list(arr, my_dict, time_map):
    for coord in arr:
        (y, x) = coord
        if (y, x) in my_dict:
            del my_dict[(y, x)]
        if (y, x) in time_map:
            del time_map[(y, x)]
    return my_dict, time_map

@njit(cache=True, nogil=True, fastmath=True)
def fill_widefield(dict_events, max_x, max_y):
    widefield = np.zeros((max_y+1, max_x+1), dtype=np.uint64)
    for key in dict_events.keys():
        widefield[key[0], key[1]] = dict_events[key][2][0]
    return widefield

@njit(nogil=True, cache=True, fastmath=True)
def detect_outlier(data):
    q1, q3 = np.percentile(data, [40, 99.99])
    iqr = q3 - q1
    lower_bound = q1
    upper_bound = q3 + (4 * iqr)
    outliers_id = []
    for id, x  in enumerate(data):
        if x <= lower_bound or x >= upper_bound:
            outliers_id.append(id)
    return outliers_id

@njit(cache=True, nogil=True, fastmath=True)
def array_to_polarity_map(arr):
    """
    Converts a structured NumPy ndarray with fields x, y, p, t into a dictionary with keys as (x, y) pairs and
    values as a nested dictionary with keys from p and corresponding values from t as a list for that coordinate pair.
    """
    dict_out = {}
    time_map = {}
    # max_len = 0
    for id in prange(len(arr)):
        key = (arr[id]["y"], arr[id]["x"])
        if key in dict_out:
            dict_out[key][arr[id]["p"]].append(arr[id]["t"])
        else:
            dict_out[key] = {
                0: List.empty_list(types.uint64), # negative
                1: List.empty_list(types.uint64), # positive
                2: List.empty_list(types.uint64), # sum of negative and positive num 
                3: List.empty_list(types.uint64), # negative num
                4: List.empty_list(types.uint64)  # positive num
            }
            dict_out[key][arr[id]["p"]].append(arr[id]["t"])
        if key in time_map:
            time_map[key][arr[id]["t"]] = arr[id]["p"]
        else:
            time_map[key] = {arr[id]["t"]: arr[id]["p"]}
        # if len(dict_out[key][1]) > max_len:
        #     max_len = len(dict_out[key][1])
        # if len(dict_out[key][0]) > max_len:
        #     max_len = len(dict_out[key][0])
    for key in dict_out.keys():
        dict_out[key][2].append(len(dict_out[key][0]) + len(dict_out[key][1]))
        dict_out[key][3].append(len(dict_out[key][0]))
        dict_out[key][4].append(len(dict_out[key][1]))
    return dict_out, time_map

In [78]:
sigma=10.5
radius=11
from joblib import Parallel, delayed
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

dict_events, time_map = array_to_polarity_map(events)
widefield = fill_widefield(dict_events, max_x, max_y)
plt.imsave(filename[:-4]+"_widefield.png", dpi=300, arr=widefield, cmap="gray", vmax=widefield.mean())
widefield_filtered = gaussian_filter(widefield, sigma=sigma, radius=radius)
useful_pixels = np.where(widefield_filtered >= np.percentile(widefield_filtered, 55), widefield, 0)
plt.imsave(filename[:-4]+"_useful_pixels.png", dpi=300, arr=useful_pixels, cmap="gray", vmax=useful_pixels.mean()*3)
dict_events, time_map = remove_coordinates(useful_pixels, dict_events, time_map)
lengths = np.asarray([np.array((val[0] ,val[1][2][0]), dtype=[("c", np.uint16, (2)), ("l", np.uint64)]) for val in list(dict_events.items())])
lengths = np.sort(lengths, order="l")
indices = detect_outlier(lengths['l'])
to_delete = lengths[indices]['c']
filtered = np.delete(lengths, indices, axis=0)
max_length = filtered[-1]['l']
dict_events, time_map = remove_coordinates_by_list(to_delete, dict_events, time_map)

In [ ]:
lengths = np.asarray([np.array((val[0] ,val[1][2][0]), dtype=[("c", np.uint16, (2)), ("l", np.uint64)]) for val in list(dict_events.items())])
lengths = np.sort(lengths, order="l")



In [5]:
times, cumsum, coordinates = create_convolved_signals(
    dict_events, coords, max_len=max_length*8, num_cores=NUM_CORES
)

starting loop


In [ ]:
print(f"Finding peaks... Elapsed time: {time.time() - start_time:.2f} seconds")
peak_list = find_peaks_parallel(
    times,
    cumsum,
    coordinates,
    NUM_CORES,
    prominence=PROMINENCE,
    interpolation_coefficient=5,
    spline_smooth=0.7,
)
peaks, prominences, on_times, coordinates_peaks = create_peak_lists(peak_list)
peaks_dict = group_timestamps_by_coordinate(
    coordinates_peaks, peaks, prominences, on_times
)

In [ ]:
unique_peaks = find_local_max_peak(
    peaks_dict, threshold=PEAK_TIME_THRESHOLD, neighbors=PEAK_NEIGHBORS
)

out_folder_localizations = filename[:-4] + "/"
temp_files_localization = out_folder_localizations + "temp_files/"
if not os.path.exists(out_folder_localizations):
    os.makedirs(out_folder_localizations)
if not os.path.exists(temp_files_localization):
    os.makedirs(temp_files_localization)

save_dict(
    unique_peaks,
    temp_files_localization
    + "unique_peaks_fwhm_"
    + str(DATASEET_FWHM)
    + "_prominence_"
    + str(PROMINENCE)
    + ".pkl",
)

In [ ]:
print(f"Generating ROIs... Elapsed time: {time.time() - start_time:.2f} seconds")
rois = generate_rois(
    unique_peaks,
    events_t_p_dict,
    roi_rad=ROI_RADIUS,
    min_x=min_x,
    min_y=min_y,
    num_cores=NUM_CORES,
    max_x=max_x,
    max_y=max_y,
)


In [ ]:

print(
    f"Performing localization... Elapsed time: {time.time() - start_time:.2f} seconds"
)
localizations = perfrom_localization_parallel(rois, dataset_FWHM=DATASEET_FWHM)

print(f"Finished! Total elapsed time: {time.time() - start_time:.2f} seconds")
np.save(
    temp_files_localization
    + "localizations_prominence_fwhm_"
    + str(DATASEET_FWHM)
    + "_prominence_"
    + str(PROMINENCE)
    + ".npy",
    localizations,
)
np.save(
    temp_files_localization
    + "rois_prominence_fwhm_"
    + str(DATASEET_FWHM)
    + "_prominence_"
    + str(PROMINENCE)
    + ".npy",
    rois,
)

In [ ]:
from scipy.ndimage import gaussian_filter


plt.figure(figsize=(15, 10))
plt.axis("off")
widefield_filtered = gaussian_filter(widefield, sigma=9.5, radius=9)
useful_pixels = np.where(widefield_filtered >= np.percentile(widefield_filtered, 60), widefield, 0)

# plt.imshow(gaussian_filter(widefield), sigma=0.8, radius=5), cmap="gray", vmax=30)
plt.imshow(np.where(widefield_filtered >= np.percentile(widefield_filtered, 50), widefield, 0), cmap="gray", vmax=100)